In [171]:
import pandas as pd

In [172]:
import os, sys

# Load and transform XLS to CSV

In [173]:
path = "./insee_xls"
for file in os.listdir(path):
    read_file = pd.read_excel (r'./insee_xls/'+file, header=2, dtype={"Code":str,"Libellé":str})
    read_file.to_csv("./insee_csv/"+file.split(".xls")[0]+".csv", index=False)

On charge les csv

In [174]:
dfsection = pd.read_csv("./insee_csv/naf2008_liste_n1.csv", dtype={"Code":str,"Libellé":str})

In [175]:
dfdivision = pd.read_csv("./insee_csv/naf2008_liste_n2.csv", dtype={"Code":str,"Libellé":str})

In [176]:
dfgroupe = pd.read_csv("./insee_csv/naf2008_liste_n3.csv", dtype={"Code":str,"Libellé":str})

In [177]:
dfclasse = pd.read_csv("./insee_csv/naf2008_liste_n4.csv", dtype={"Code":str,"Libellé":str})

In [178]:
dfsousclasse = pd.read_csv("./insee_csv/naf2008_liste_n5.csv", dtype={"Code":str,"Libellé":str})

On renomme les colonnes

In [179]:
dfsection = dfsection.rename(columns={'Code':'code_section','Libellé':'libelle_section'})

In [180]:
dfdivision = dfdivision.rename(columns={'Code':'code_division','Libellé':'libelle_division'})

In [181]:
dfgroupe = dfgroupe.rename(columns={'Code':'code_groupe','Libellé':'libelle_groupe'})

In [182]:
dfclasse = dfclasse.rename(columns={'Code':'code_classe','Libellé':'libelle_classe'})

In [183]:
dfsousclasse = dfsousclasse.rename(columns={'Code':'code_sous_classe','Libellé':'libelle_sous_classe'})

On ajoute une colonne "short" sans les points

In [184]:
dfgroupe['code_groupe_short'] = dfgroupe['code_groupe'].apply(lambda x: x.replace(".",""))

In [185]:
dfclasse['code_classe_short'] = dfclasse['code_classe'].apply(lambda x: x.replace(".",""))

In [186]:
dfsousclasse['code_sous_classe_short'] = dfsousclasse['code_sous_classe'].apply(lambda x: x.replace(".",""))

# Manipulation des niveaux

In [187]:
dfsousclasse['code_classe'] = dfsousclasse['code_sous_classe'].apply(lambda x: x[:5])

In [188]:
dfintermediary = pd.merge(dfsousclasse,dfclasse,on='code_classe',how='left')

In [189]:
dfintermediary['code_groupe'] = dfintermediary['code_classe'].apply(lambda x: x[:4])

In [190]:
dfintermediary = pd.merge(dfintermediary,dfgroupe,on='code_groupe',how='left')

In [191]:
dfintermediary['code_division'] = dfintermediary['code_groupe'].apply(lambda x: x[:2])

In [192]:
dfintermediary = pd.merge(dfintermediary,dfdivision,on='code_division',how='left')

In [193]:
dfintermediary

,code_sous_classe,libelle_sous_classe,code_sous_classe_short,code_classe,libelle_classe,code_classe_short,code_groupe,libelle_groupe,code_groupe_short,code_division,libelle_division
0,01.11Z,"Culture de céréales (à l'exception du riz), de...",0111Z,01.11,"Culture de céréales (à l'exception du riz), de...",0111,01.1,Cultures non permanentes,011,01,"Culture et production animale, chasse et servi..."
1,01.12Z,Culture du riz,0112Z,01.12,Culture du riz,0112,01.1,Cultures non permanentes,011,01,"Culture et production animale, chasse et servi..."
2,01.13Z,"Culture de légumes, de melons, de racines et d...",0113Z,01.13,"Culture de légumes, de melons, de racines et d...",0113,01.1,Cultures non permanentes,011,01,"Culture et production animale, chasse et servi..."
3,01.14Z,Culture de la canne à sucre,0114Z,01.14,Culture de la canne à sucre,0114,01.1,Cultures non permanentes,011,01,"Culture et production animale, chasse et servi..."
4,01.15Z,Culture du tabac,0115Z,01.15,Culture du tabac,0115,01.1,Cultures non permanentes,011,01,"Culture et production animale, chasse et servi..."
...,...,...,...,...,...,...,...,...,...,...,...
727,96.09Z,Autres services personnels n.c.a.,9609Z,96.09,Autres services personnels n.c.a.,9609,96.0,Autres services personnels,960,96,Autres services personnels
728,97.00Z,Activités des ménages en tant qu'employeurs de...,9700Z,97.00,Activités des ménages en tant qu'employeurs de...,9700,97.0,Activités des ménages en tant qu'employeurs de...,970,97,Activités des ménages en tant qu'employeurs de...
729,98.10Z,Activités indifférenciées des ménages en tant ...,9810Z,98.10,Activités indifférenciées des ménages en tant ...,9810,98.1,Activités indifférenciées des ménages en tant ...,981,98,Activités indifférenciées des ménages en tant ...
730,98.20Z,Activités indifférenciées des ménages en tant ...,9820Z,98.20,Activités indifférenciées des ménages en tant ...,9820,98.2,Activités indifférenciées des ménages en tant ...,982,98,Activités indifférenciées des ménages en tant ...


On charge le fichier excel des 5 niveaux pour avoir la correspondance division <> section

In [194]:
df5niv = pd.read_excel("./insee_xls/naf2008_5_niveaux.xls",dtype={'NIV2':str,'NIV1':str})

In [195]:
df5niv = df5niv[['NIV2','NIV1']]

In [196]:
df5niv = df5niv.rename(columns={'NIV2':'code_division','NIV1':'code_section'})

In [197]:
df5niv = df5niv.drop

,code_division,code_section
0,01,A
1,01,A
2,01,A
3,01,A
4,01,A
...,...,...
727,96,S
728,97,T
729,98,T
730,98,T


In [198]:
dfintermediary = pd.merge(dfintermediary,df5niv,on='code_division',how='left')

In [199]:
dfintermediary

,code_sous_classe,libelle_sous_classe,code_sous_classe_short,code_classe,libelle_classe,code_classe_short,code_groupe,libelle_groupe,code_groupe_short,code_division,libelle_division,code_section
0,01.11Z,"Culture de céréales (à l'exception du riz), de...",0111Z,01.11,"Culture de céréales (à l'exception du riz), de...",0111,01.1,Cultures non permanentes,011,01,"Culture et production animale, chasse et servi...",A
1,01.11Z,"Culture de céréales (à l'exception du riz), de...",0111Z,01.11,"Culture de céréales (à l'exception du riz), de...",0111,01.1,Cultures non permanentes,011,01,"Culture et production animale, chasse et servi...",A
2,01.11Z,"Culture de céréales (à l'exception du riz), de...",0111Z,01.11,"Culture de céréales (à l'exception du riz), de...",0111,01.1,Cultures non permanentes,011,01,"Culture et production animale, chasse et servi...",A
3,01.11Z,"Culture de céréales (à l'exception du riz), de...",0111Z,01.11,"Culture de céréales (à l'exception du riz), de...",0111,01.1,Cultures non permanentes,011,01,"Culture et production animale, chasse et servi...",A
4,01.11Z,"Culture de céréales (à l'exception du riz), de...",0111Z,01.11,"Culture de céréales (à l'exception du riz), de...",0111,01.1,Cultures non permanentes,011,01,"Culture et production animale, chasse et servi...",A
...,...,...,...,...,...,...,...,...,...,...,...,...
14037,98.10Z,Activités indifférenciées des ménages en tant ...,9810Z,98.10,Activités indifférenciées des ménages en tant ...,9810,98.1,Activités indifférenciées des ménages en tant ...,981,98,Activités indifférenciées des ménages en tant ...,T
14038,98.10Z,Activités indifférenciées des ménages en tant ...,9810Z,98.10,Activités indifférenciées des ménages en tant ...,9810,98.1,Activités indifférenciées des ménages en tant ...,981,98,Activités indifférenciées des ménages en tant ...,T
14039,98.20Z,Activités indifférenciées des ménages en tant ...,9820Z,98.20,Activités indifférenciées des ménages en tant ...,9820,98.2,Activités indifférenciées des ménages en tant ...,982,98,Activités indifférenciées des ménages en tant ...,T
14040,98.20Z,Activités indifférenciées des ménages en tant ...,9820Z,98.20,Activités indifférenciées des ménages en tant ...,9820,98.2,Activités indifférenciées des ménages en tant ...,982,98,Activités indifférenciées des ménages en tant ...,T


In [167]:
dfintermediary = pd.merge(dfintermediary,dfsection,on='code_section',how='left')

In [169]:
dfintermediary

,code_sous_classe,libelle_sous_classe,code_sous_classe_short,code_classe,libelle_classe,code_classe_short,code_groupe,libelle_groupe,code_groupe_short,code_division,libelle_division,code_section,libelle_section
0,01.11Z,"Culture de céréales (à l'exception du riz), de...",0111Z,01.11,"Culture de céréales (à l'exception du riz), de...",0111,01.1,Cultures non permanentes,011,01,"Culture et production animale, chasse et servi...",A,"Agriculture, sylviculture et pêche"
1,01.11Z,"Culture de céréales (à l'exception du riz), de...",0111Z,01.11,"Culture de céréales (à l'exception du riz), de...",0111,01.1,Cultures non permanentes,011,01,"Culture et production animale, chasse et servi...",A,"Agriculture, sylviculture et pêche"
2,01.11Z,"Culture de céréales (à l'exception du riz), de...",0111Z,01.11,"Culture de céréales (à l'exception du riz), de...",0111,01.1,Cultures non permanentes,011,01,"Culture et production animale, chasse et servi...",A,"Agriculture, sylviculture et pêche"
3,01.11Z,"Culture de céréales (à l'exception du riz), de...",0111Z,01.11,"Culture de céréales (à l'exception du riz), de...",0111,01.1,Cultures non permanentes,011,01,"Culture et production animale, chasse et servi...",A,"Agriculture, sylviculture et pêche"
4,01.11Z,"Culture de céréales (à l'exception du riz), de...",0111Z,01.11,"Culture de céréales (à l'exception du riz), de...",0111,01.1,Cultures non permanentes,011,01,"Culture et production animale, chasse et servi...",A,"Agriculture, sylviculture et pêche"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14037,98.10Z,Activités indifférenciées des ménages en tant ...,9810Z,98.10,Activités indifférenciées des ménages en tant ...,9810,98.1,Activités indifférenciées des ménages en tant ...,981,98,Activités indifférenciées des ménages en tant ...,T,Activités des ménages en tant qu'employeurs ; ...
14038,98.10Z,Activités indifférenciées des ménages en tant ...,9810Z,98.10,Activités indifférenciées des ménages en tant ...,9810,98.1,Activités indifférenciées des ménages en tant ...,981,98,Activités indifférenciées des ménages en tant ...,T,Activités des ménages en tant qu'employeurs ; ...
14039,98.20Z,Activités indifférenciées des ménages en tant ...,9820Z,98.20,Activités indifférenciées des ménages en tant ...,9820,98.2,Activités indifférenciées des ménages en tant ...,982,98,Activités indifférenciées des ménages en tant ...,T,Activités des ménages en tant qu'employeurs ; ...
14040,98.20Z,Activités indifférenciées des ménages en tant ...,9820Z,98.20,Activités indifférenciées des ménages en tant ...,9820,98.2,Activités indifférenciées des ménages en tant ...,982,98,Activités indifférenciées des ménages en tant ...,T,Activités des ménages en tant qu'employeurs ; ...


In [170]:
dfintermediary.to_csv("naf_complet.csv",index=False)